In [1]:
import glob
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
import re

### Preprocessing

In [3]:
authors = ["M._Alessandri_di_Chidazzu", "Petru_Rocca", "Martinu_Appinzapalu"]
disputed = ['P._di_B.']

In [4]:
names = authors + disputed

In [5]:
corpus = {}

for name in names:
    files = glob.glob(f"./tests/{name}/*.txt")
    
    text = []
    for file in files:
        if re.search(r"_co_", file):
            with open(file, 'r') as f:
                content = f.readlines()
            text.extend(content)
    
    tokenizer = RegexpTokenizer('\w+')
    text = ' '.join(text)
    token = tokenizer.tokenize(text)
    
    corpus[name] = token

### Computing

In [6]:
for author in authors:
    joint_corpus = (corpus[author] +
                    corpus["P._di_B."])
    joint_freq_dist = FreqDist(joint_corpus)
    most_common = list(joint_freq_dist.most_common(500))

    author_share = (len(corpus[author])
                    / len(joint_corpus))
    
    joint_corpus = (corpus[author] +
                    corpus["P._di_B."])
    joint_freq_dist = FreqDist(joint_corpus)
    most_common = list(joint_freq_dist.most_common(500))
    
    author_share = (len(corpus[author])
                    / len(joint_corpus))
    
    chisquared = 0
    for word,joint_count in most_common:

        author_count = corpus[author].count(word)
        disputed_count = corpus["P._di_B."].count(word)

        expected_author_count = joint_count * author_share
        expected_disputed_count = joint_count * (1-author_share)
        
        if expected_author_count != 0 and expected_disputed_count != 0:

            chisquared += ((author_count-expected_author_count) *
                           (author_count-expected_author_count) /
                           expected_author_count)

            chisquared += ((disputed_count-expected_disputed_count) *
                           (disputed_count-expected_disputed_count)
                           / expected_disputed_count)

    print("The Chi-squared statistic for candidate", author, "is", chisquared)

The Chi-squared statistic for candidate M._Alessandri_di_Chidazzu is 4122.627001466433
The Chi-squared statistic for candidate Petru_Rocca is 1548.1498346557494
The Chi-squared statistic for candidate Martinu_Appinzapalu is 3764.446135432509
